In [1]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [2]:
with open('clean_bb_comments.json') as json_file:  #first 1mill bb comments
    data = json.load(json_file)

In [3]:

bb_tok = Tokenizer(oov_token='<unk>')
bb_tok.fit_on_texts(data)

In [4]:
len(bb_tok.word_index)

50002

In [5]:
with open('bb-v5-tok.pickle', 'wb') as handle:
    pickle.dump(bb_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [6]:
with open('bb-500k-tok.pickle', 'rb') as handle:
    bb_tok = pickle.load(handle)

In [7]:


train_data = data

seqs = bb_tok.texts_to_sequences(train_data)
less_20_seqs = list(filter(lambda x: len(x) <20, seqs))

In [8]:
len(less_20_seqs)

654993

In [ ]:
MAXLEN = 19
VOCAB = int(len(bb_tok.word_index))
SEED = 42

sequences = pad_sequences(less_20_seqs, maxlen=MAXLEN, padding='pre')

print('Max Seq Len: %d' % MAXLEN)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=SEED)


#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [ ]:


#NN definition
model = Sequential()
model.add(Embedding(VOCAB, 300, input_length=MAXLEN-1))
model.add(LSTM(500))



model.add(Dense(VOCAB, activation='softmax'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           43245300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 500)               1602000   
_________________________________________________________________
dense_1 (Dense)              (None, 144151)            72219651  
Total params: 117,066,951
Trainable params: 117,066,951
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


In [ ]:

model_json = model.to_json()
with open("models/bb-model-v5.json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/bb-v5/best-weights.hdf5"
checkpoint = ModelCheckpoint(fpath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')


BS = 100
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('bb-modelv5'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X_train,y_train,batch_size=BS, validation_split=.005, epochs=20, callbacks=callback_lst, verbose=1)



Instructions for updating:
Use tf.cast instead.
Train on 553960 samples, validate on 2784 samples
Epoch 1/20
553960/553960 [==============================] - 825s 1ms/step - loss: 8.6359 - acc: 0.0420 - val_loss: 8.5017 - val_acc: 0.0413

Epoch 00001: val_acc improved from -inf to 0.04131, saving model to weights/bb-v5/best-weights.hdf5
Epoch 2/20
101100/553960 [====>.........................] - ETA: 12:49 - loss: 8.4979 - acc: 0.0431